In [1]:
!nvidia-smi
!nvcc --version


Sun May 25 16:49:33 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   48C    P8              9W /   70W |       1MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
%%writefile rt_simulation.cu
#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include <sys/stat.h>
#include <cuda_runtime.h>

#define Nx 512

#define Ny 1024
#define Lx 1.0
#define Ly 2.0
#define dx (Lx / Nx)
#define dy (Ly / Ny)

#define gamma 1.4
#define ga -10.0
#define C 0.2
#define save_interval 1000
#define BLOCK_SIZE_X 16
#define BLOCK_SIZE_Y 16

// Error checking macro
#define cudaCheckError() { \
    cudaError_t err = cudaGetLastError(); \
    if(err != cudaSuccess) { \
        printf("CUDA error: %s at line %d\n", cudaGetErrorString(err), __LINE__); \
        exit(EXIT_FAILURE); \
    } \
}

void update_diffusion_coeffs(double dt, double *k1, double *k2, double *k3) {
    *k1 = 0.0125 * (dx * dx) / (2 * dt);
    *k2 = 0.125 * (dx * dx) / (2 * dt);
    *k3 = 0.0125 * (dx * dx) / (2 * dt);
}

__global__ void initialize_grid_kernel(double *r, double *ru, double *rv, double *e, double *p) {
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    int j = blockIdx.y * blockDim.y + threadIdx.y;

    if (i < Nx && j < Ny) {
        double y = (j + 0.5) * dy;
        double v_pert = 0.0;

        // Initialize density based on position
        if (y >= Ly / 2) {
            r[j * Nx + i] = 2.0;
        } else {
            r[j * Nx + i] = 1.0;
        }

        // Add perturbation at the interface
        if (fabs(y - Ly / 2) <= 0.05) {
            unsigned int seed = i * 1723 + j * 93241;
            seed = (seed * 196314165) + 907633515;
            double rand_val = (seed % 10000) / 5000.0 - 1.0;
            v_pert = 1e-3 * rand_val;
        }

        // Initialize momentum and energy
        double u = 0.0;
        double v = v_pert;
        p[j * Nx + i] = 40.0 + r[j * Nx + i] * ga * (y - Ly / 2);
        ru[j * Nx + i] = r[j * Nx + i] * u;
        rv[j * Nx + i] = r[j * Nx + i] * v;
        e[j * Nx + i] = p[j * Nx + i] / (gamma - 1) + 0.5 * r[j * Nx + i] * (u * u + v * v);
    }
}

void initialize_grid(double *r, double *ru, double *rv, double *e, double *p) {
    dim3 blockDim(BLOCK_SIZE_X, BLOCK_SIZE_Y);
    dim3 gridDim((Nx + blockDim.x - 1) / blockDim.x, (Ny + blockDim.y - 1) / blockDim.y);

    initialize_grid_kernel<<<gridDim, blockDim>>>(r, ru, rv, e, p);
    cudaDeviceSynchronize();
    cudaCheckError();
}

__global__ void compute_dt_kernel(double *r, double *ru, double *rv, double *p, double *max_speeds) {
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    int j = blockIdx.y * blockDim.y + threadIdx.y;

    extern __shared__ double shared_max[];

    shared_max[threadIdx.y * blockDim.x + threadIdx.x] = 0.0;

    if (i < Nx && j < Ny) {
        double u = 0.0, v = 0.0, c = 0.0;

        if (r[j * Nx + i] > 1e-10) {
            u = ru[j * Nx + i] / r[j * Nx + i];
            v = rv[j * Nx + i] / r[j * Nx + i];
        }

        c = sqrt(gamma * p[j * Nx + i] / r[j * Nx + i]);
        double speed = fmax(fmax(fabs(u), fabs(v)), c);

        shared_max[threadIdx.y * blockDim.x + threadIdx.x] = speed;
    }

    __syncthreads();

    for (unsigned int s = blockDim.x * blockDim.y / 2; s > 0; s >>= 1) {
        if (threadIdx.y * blockDim.x + threadIdx.x < s) {
            shared_max[threadIdx.y * blockDim.x + threadIdx.x] = fmax(
                shared_max[threadIdx.y * blockDim.x + threadIdx.x],
                shared_max[(threadIdx.y * blockDim.x + threadIdx.x) + s]
            );
        }
        __syncthreads();
    }

    if (threadIdx.y == 0 && threadIdx.x == 0) {
        max_speeds[blockIdx.y * gridDim.x + blockIdx.x] = shared_max[0];
    }
}

double compute_dt(double *r, double *ru, double *rv, double *p) {
    dim3 blockDim(BLOCK_SIZE_X, BLOCK_SIZE_Y);
    dim3 gridDim((Nx + blockDim.x - 1) / blockDim.x, (Ny + blockDim.y - 1) / blockDim.y);

    int blocks_count = gridDim.x * gridDim.y;
    double *d_max_speeds, *h_max_speeds;

    cudaMalloc(&d_max_speeds, blocks_count * sizeof(double));
    h_max_speeds = (double*)malloc(blocks_count * sizeof(double));
    cudaCheckError();

    size_t shared_mem_size = BLOCK_SIZE_X * BLOCK_SIZE_Y * sizeof(double);
    compute_dt_kernel<<<gridDim, blockDim, shared_mem_size>>>(r, ru, rv, p, d_max_speeds);
    cudaDeviceSynchronize();
    cudaCheckError();

    cudaMemcpy(h_max_speeds, d_max_speeds, blocks_count * sizeof(double), cudaMemcpyDeviceToHost);
    cudaCheckError();

    double max_speed = 0.0;
    for (int i = 0; i < blocks_count; i++) {
        if (h_max_speeds[i] > max_speed) {
            max_speed = h_max_speeds[i];
        }
    }

    cudaFree(d_max_speeds);
    free(h_max_speeds);

    return C * fmin(dx, dy) / (max_speed > 1e-10 ? max_speed : 1e-10);
}

__global__ void compute_derivatives_kernel(double *f, double *df_dx, double *df_dy,
                                          double *d2f_dx2, double *d2f_dy2) {
    int i = blockIdx.x * blockDim.x + threadIdx.x + 1;
    int j = blockIdx.y * blockDim.y + threadIdx.y + 1;

    if (i < Nx - 1 && j < Ny - 1) {
        df_dx[j * Nx + i] = (f[j * Nx + (i + 1)] - f[j * Nx + (i - 1)]) / (2 * dx);
        df_dy[j * Nx + i] = (f[(j + 1) * Nx + i] - f[(j - 1) * Nx + i]) / (2 * dy);
        d2f_dx2[j * Nx + i] = (f[j * Nx + (i + 1)] - 2 * f[j * Nx + i] + f[j * Nx + (i - 1)]) / (dx * dx);
        d2f_dy2[j * Nx + i] = (f[(j + 1) * Nx + i] - 2 * f[j * Nx + i] + f[(j - 1) * Nx + i]) / (dy * dy);
    }
}

__global__ void compute_velocities_kernel(double *r, double *ru, double *rv,
                                         double *u, double *v) {
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    int j = blockIdx.y * blockDim.y + threadIdx.y;

    if (i < Nx && j < Ny) {
        if (r[j * Nx + i] > 1e-10) {
            u[j * Nx + i] = ru[j * Nx + i] / r[j * Nx + i];
            v[j * Nx + i] = rv[j * Nx + i] / r[j * Nx + i];
        } else {
            u[j * Nx + i] = 0.0;
            v[j * Nx + i] = 0.0;
        }
    }
}

__global__ void compute_fluxes_kernel(double *r, double *ru, double *rv, double *e, double *p,
                                     double *u, double *v,
                                     double *ruu, double *ruv, double *rvv,
                                     double *eu_p, double *ev_p) {
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    int j = blockIdx.y * blockDim.y + threadIdx.y;

    if (i < Nx && j < Ny) {
        ruu[j * Nx + i] = ru[j * Nx + i] * u[j * Nx + i];
        ruv[j * Nx + i] = ru[j * Nx + i] * v[j * Nx + i];
        rvv[j * Nx + i] = rv[j * Nx + i] * v[j * Nx + i];
        eu_p[j * Nx + i] = u[j * Nx + i] * (e[j * Nx + i] + p[j * Nx + i]);
        ev_p[j * Nx + i] = v[j * Nx + i] * (e[j * Nx + i] + p[j * Nx + i]);
    }
}

__global__ void compute_rhs_kernel(double *r, double *ru, double *rv, double *e,
                                  double *p, double k1, double k2, double k3,
                                  double *dr_dx, double *dr_dy, double *d2r_dx2, double *d2r_dy2,
                                  double *dru_dx, double *dru_dy, double *d2ru_dx2, double *d2ru_dy2,
                                  double *drv_dx, double *drv_dy, double *d2rv_dx2, double *d2rv_dy2,
                                  double *de_dx, double *de_dy, double *d2e_dx2, double *d2e_dy2,
                                  double *dp_dx, double *dp_dy,
                                  double *druu_dx, double *druv_dx, double *druv_dy, double *drvv_dy,
                                  double *deup_dx, double *devp_dy,
                                  double *u, double *v,
                                  double *rhs_r, double *rhs_ru, double *rhs_rv, double *rhs_e) {
    int i = blockIdx.x * blockDim.x + threadIdx.x + 1;
    int j = blockIdx.y * blockDim.y + threadIdx.y + 1;

    if (i < Nx - 1 && j < Ny - 1) {
        rhs_r[j * Nx + i] = -(dru_dx[j * Nx + i] + drv_dy[j * Nx + i]) + k1 * (d2r_dx2[j * Nx + i] + d2r_dy2[j * Nx + i]);
        rhs_ru[j * Nx + i] = -(druu_dx[j * Nx + i] + druv_dy[j * Nx + i]) - dp_dx[j * Nx + i] + k2 * d2ru_dx2[j * Nx + i];
        rhs_rv[j * Nx + i] = -(druv_dx[j * Nx + i] + drvv_dy[j * Nx + i]) - dp_dy[j * Nx + i] + ga * r[j * Nx + i] + k2 * d2rv_dy2[j * Nx + i];
        rhs_e[j * Nx + i] = -(deup_dx[j * Nx + i] + devp_dy[j * Nx + i]) + ga * r[j * Nx + i] * v[j * Nx + i] + k3 * (d2e_dx2[j * Nx + i] + d2e_dy2[j * Nx + i]);
    }
}

void compute_rhs(double *r, double *ru, double *rv, double *e,
                double *p, double k1, double k2, double k3,
                double *rhs_r, double *rhs_ru, double *rhs_rv, double *rhs_e) {
    double *u, *v;
    double *ruu, *ruv, *rvv, *eu_p, *ev_p;
    double *dr_dx, *dr_dy, *d2r_dx2, *d2r_dy2;
    double *dru_dx, *dru_dy, *d2ru_dx2, *d2ru_dy2;
    double *drv_dx, *drv_dy, *d2rv_dx2, *d2rv_dy2;
    double *de_dx, *de_dy, *d2e_dx2, *d2e_dy2;
    double *dp_dx, *dp_dy, *dummy1, *dummy2;
    double *druu_dx, *druv_dx, *druv_dy, *drvv_dy;
    double *deup_dx, *devp_dy;

    size_t array_size = Nx * Ny * sizeof(double);

    cudaMalloc(&u, array_size);
    cudaMalloc(&v, array_size);
    cudaMalloc(&ruu, array_size);
    cudaMalloc(&ruv, array_size);
    cudaMalloc(&rvv, array_size);
    cudaMalloc(&eu_p, array_size);
    cudaMalloc(&ev_p, array_size);
    cudaMalloc(&dr_dx, array_size);
    cudaMalloc(&dr_dy, array_size);
    cudaMalloc(&d2r_dx2, array_size);
    cudaMalloc(&d2r_dy2, array_size);
    cudaMalloc(&dru_dx, array_size);
    cudaMalloc(&dru_dy, array_size);
    cudaMalloc(&d2ru_dx2, array_size);
    cudaMalloc(&d2ru_dy2, array_size);
    cudaMalloc(&drv_dx, array_size);
    cudaMalloc(&drv_dy, array_size);
    cudaMalloc(&d2rv_dx2, array_size);
    cudaMalloc(&d2rv_dy2, array_size);
    cudaMalloc(&de_dx, array_size);
    cudaMalloc(&de_dy, array_size);
    cudaMalloc(&d2e_dx2, array_size);
    cudaMalloc(&d2e_dy2, array_size);
    cudaMalloc(&dp_dx, array_size);
    cudaMalloc(&dp_dy, array_size);
    cudaMalloc(&dummy1, array_size);
    cudaMalloc(&dummy2, array_size);
    cudaMalloc(&druu_dx, array_size);
    cudaMalloc(&druv_dx, array_size);
    cudaMalloc(&druv_dy, array_size);
    cudaMalloc(&drvv_dy, array_size);
    cudaMalloc(&deup_dx, array_size);
    cudaMalloc(&devp_dy, array_size);
    cudaCheckError();

    dim3 blockDim(BLOCK_SIZE_X, BLOCK_SIZE_Y);
    dim3 gridDim((Nx + blockDim.x - 1) / blockDim.x, (Ny + blockDim.y - 1) / blockDim.y);
    dim3 gridDimInt((Nx - 2 + blockDim.x - 1) / blockDim.x, (Ny - 2 + blockDim.y - 1) / blockDim.y);

    compute_velocities_kernel<<<gridDim, blockDim>>>(r, ru, rv, u, v);
    cudaDeviceSynchronize();
    cudaCheckError();

    compute_fluxes_kernel<<<gridDim, blockDim>>>(r, ru, rv, e, p, u, v, ruu, ruv, rvv, eu_p, ev_p);
    cudaDeviceSynchronize();
    cudaCheckError();

    compute_derivatives_kernel<<<gridDimInt, blockDim>>>(r, dr_dx, dr_dy, d2r_dx2, d2r_dy2);
    compute_derivatives_kernel<<<gridDimInt, blockDim>>>(ru, dru_dx, dru_dy, d2ru_dx2, d2ru_dy2);
    compute_derivatives_kernel<<<gridDimInt, blockDim>>>(rv, drv_dx, drv_dy, d2rv_dx2, d2rv_dy2);
    compute_derivatives_kernel<<<gridDimInt, blockDim>>>(e, de_dx, de_dy, d2e_dx2, d2e_dy2);
    compute_derivatives_kernel<<<gridDimInt, blockDim>>>(p, dp_dx, dp_dy, dummy1, dummy2);
    compute_derivatives_kernel<<<gridDimInt, blockDim>>>(ruu, druu_dx, dummy1, dummy2, dummy2);
    compute_derivatives_kernel<<<gridDimInt, blockDim>>>(ruv, druv_dx, druv_dy, dummy1, dummy2);
    compute_derivatives_kernel<<<gridDimInt, blockDim>>>(rvv, dummy1, drvv_dy, dummy2, dummy2);
    compute_derivatives_kernel<<<gridDimInt, blockDim>>>(eu_p, deup_dx, dummy1, dummy2, dummy2);
    compute_derivatives_kernel<<<gridDimInt, blockDim>>>(ev_p, dummy1, devp_dy, dummy2, dummy2);
    cudaDeviceSynchronize();
    cudaCheckError();

    compute_rhs_kernel<<<gridDimInt, blockDim>>>(r, ru, rv, e, p, k1, k2, k3,
                                               dr_dx, dr_dy, d2r_dx2, d2r_dy2,
                                               dru_dx, dru_dy, d2ru_dx2, d2ru_dy2,
                                               drv_dx, drv_dy, d2rv_dx2, d2rv_dy2,
                                               de_dx, de_dy, d2e_dx2, d2e_dy2,
                                               dp_dx, dp_dy,
                                               druu_dx, druv_dx, druv_dy, drvv_dy,
                                               deup_dx, devp_dy,
                                               u, v,
                                               rhs_r, rhs_ru, rhs_rv, rhs_e);
    cudaDeviceSynchronize();
    cudaCheckError();

    cudaFree(u);
    cudaFree(v);
    cudaFree(ruu);
    cudaFree(ruv);
    cudaFree(rvv);
    cudaFree(eu_p);
    cudaFree(ev_p);
    cudaFree(dr_dx);
    cudaFree(dr_dy);
    cudaFree(d2r_dx2);
    cudaFree(d2r_dy2);
    cudaFree(dru_dx);
    cudaFree(dru_dy);
    cudaFree(d2ru_dx2);
    cudaFree(d2ru_dy2);
    cudaFree(drv_dx);
    cudaFree(drv_dy);
    cudaFree(d2rv_dx2);
    cudaFree(d2rv_dy2);
    cudaFree(de_dx);
    cudaFree(de_dy);
    cudaFree(d2e_dx2);
    cudaFree(d2e_dy2);
    cudaFree(dp_dx);
    cudaFree(dp_dy);
    cudaFree(dummy1);
    cudaFree(dummy2);
    cudaFree(druu_dx);
    cudaFree(druv_dx);
    cudaFree(druv_dy);
    cudaFree(drvv_dy);
    cudaFree(deup_dx);
    cudaFree(devp_dy);
}

__global__ void apply_boundary_conditions_kernel(double *r, double *ru, double *rv,
                                               double *e, double *p) {
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    int j = blockIdx.y * blockDim.y + threadIdx.y;

    if (j < Ny && (i == 0 || i == Nx - 1)) {
        if (i == 0) {
            r[j * Nx + i] = r[j * Nx + (Nx - 2)];
            ru[j * Nx + i] = ru[j * Nx + (Nx - 2)];
            rv[j * Nx + i] = rv[j * Nx + (Nx - 2)];
            e[j * Nx + i] = e[j * Nx + (Nx - 2)];
            p[j * Nx + i] = p[j * Nx + (Nx - 2)];
        } else {
            r[j * Nx + i] = r[j * Nx + 1];
            ru[j * Nx + i] = ru[j * Nx + 1];
            rv[j * Nx + i] = rv[j * Nx + 1];
            e[j * Nx + i] = e[j * Nx + 1];
            p[j * Nx + i] = p[j * Nx + 1];
        }
    }

    if (i < Nx && (j == 0 || j == Ny - 1)) {
        if (j == 0) {
            rv[j * Nx + i] = 0;
            r[j * Nx + i] = r[(j + 1) * Nx + i];
            ru[j * Nx + i] = ru[(j + 1) * Nx + i];
            p[j * Nx + i] = p[(j + 1) * Nx + i];

            double u = 0.0, v = 0.0;
            if (r[j * Nx + i] > 1e-10) {
                u = ru[j * Nx + i] / r[j * Nx + i];
                v = rv[j * Nx + i] / r[j * Nx + i];
            }
            e[j * Nx + i] = p[j * Nx + i] / (gamma - 1) +
                         0.5 * r[j * Nx + i] * (u * u + v * v);
        } else {
            rv[j * Nx + i] = 0;
            r[j * Nx + i] = r[(j - 1) * Nx + i];
            ru[j * Nx + i] = ru[(j - 1) * Nx + i];
            p[j * Nx + i] = p[(j - 1) * Nx + i];

            double u = 0.0, v = 0.0;
            if (r[j * Nx + i] > 1e-10) {
                u = ru[j * Nx + i] / r[j * Nx + i];
                v = rv[j * Nx + i] / r[j * Nx + i];
            }
            e[j * Nx + i] = p[j * Nx + i] / (gamma - 1) +
                         0.5 * r[j * Nx + i] * (u * u + v * v);
        }
    }
}

void apply_boundary_conditions(double *r, double *ru, double *rv, double *e, double *p) {
    dim3 blockDim(BLOCK_SIZE_X, BLOCK_SIZE_Y);
    dim3 gridDim((Nx + blockDim.x - 1) / blockDim.x, (Ny + blockDim.y - 1) / blockDim.y);

    apply_boundary_conditions_kernel<<<gridDim, blockDim>>>(r, ru, rv, e, p);
    cudaDeviceSynchronize();
    cudaCheckError();
}

__global__ void update_pressure_kernel(double *r, double *ru, double *rv,
                                     double *e, double *p) {
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    int j = blockIdx.y * blockDim.y + threadIdx.y;

    if (i < Nx && j < Ny) {
        double u = 0.0, v = 0.0;
        if (r[j * Nx + i] > 1e-10) {
            u = ru[j * Nx + i] / r[j * Nx + i];
            v = rv[j * Nx + i] / r[j * Nx + i];
        }

        p[j * Nx + i] = (gamma - 1) * (e[j * Nx + i] - 0.5 * r[j * Nx + i] * (u * u + v * v));
        if (p[j * Nx + i] < 1e-10) {
            p[j * Nx + i] = 1e-10;
        }
    }
}

void update_pressure(double *r, double *ru, double *rv, double *e, double *p) {
    dim3 blockDim(BLOCK_SIZE_X, BLOCK_SIZE_Y);
    dim3 gridDim((Nx + blockDim.x - 1) / blockDim.x, (Ny + blockDim.y - 1) / blockDim.y);

    update_pressure_kernel<<<gridDim, blockDim>>>(r, ru, rv, e, p);
    cudaDeviceSynchronize();
    cudaCheckError();
}

__global__ void euler_step_kernel(double *r, double *ru, double *rv, double *e,
                                double *rhs_r, double *rhs_ru, double *rhs_rv, double *rhs_e,
                                double *r_new, double *ru_new, double *rv_new, double *e_new,
                                double dt) {
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    int j = blockIdx.y * blockDim.y + threadIdx.y;

    if (i < Nx && j < Ny) {
        r_new[j * Nx + i] = r[j * Nx + i] + dt * rhs_r[j * Nx + i];
        ru_new[j * Nx + i] = ru[j * Nx + i] + dt * rhs_ru[j * Nx + i];
        rv_new[j * Nx + i] = rv[j * Nx + i] + dt * rhs_rv[j * Nx + i];
        e_new[j * Nx + i] = e[j * Nx + i] + dt * rhs_e[j * Nx + i];
    }
}

__global__ void update_state_kernel(double *r, double *ru, double *rv, double *e, double *p,
                                  double *r_new, double *ru_new, double *rv_new, double *e_new, double *p_new) {
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    int j = blockIdx.y * blockDim.y + threadIdx.y;

    if (i < Nx && j < Ny) {
        r[j * Nx + i] = r_new[j * Nx + i];
        ru[j * Nx + i] = ru_new[j * Nx + i];
        rv[j * Nx + i] = rv_new[j * Nx + i];
        e[j * Nx + i] = e_new[j * Nx + i];
        p[j * Nx + i] = p_new[j * Nx + i];
    }
}

__global__ void rk2_step_kernel1(double *r, double *ru, double *rv, double *e,
                                double *rhs_r, double *rhs_ru, double *rhs_rv, double *rhs_e,
                                double *r_star, double *ru_star, double *rv_star, double *e_star,
                                double dt) {
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    int j = blockIdx.y * blockDim.y + threadIdx.y;

    if (i < Nx && j < Ny) {
        r_star[j * Nx + i] = r[j * Nx + i] + dt * rhs_r[j * Nx + i];
        ru_star[j * Nx + i] = ru[j * Nx + i] + dt * rhs_ru[j * Nx + i];
        rv_star[j * Nx + i] = rv[j * Nx + i] + dt * rhs_rv[j * Nx + i];
        e_star[j * Nx + i] = e[j * Nx + i] + dt * rhs_e[j * Nx + i];
    }
}

__global__ void rk2_step_kernel2(double *r, double *ru, double *rv, double *e,
                                double *r_star, double *ru_star, double *rv_star, double *e_star,
                                double *rhs_r_star, double *rhs_ru_star, double *rhs_rv_star, double *rhs_e_star,
                                double *r_new, double *ru_new, double *rv_new, double *e_new,
                                double dt) {
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    int j = blockIdx.y * blockDim.y + threadIdx.y;

    if (i < Nx && j < Ny) {
        r_new[j * Nx + i] = 0.5 * r[j * Nx + i] + 0.5 * (r_star[j * Nx + i] + dt * rhs_r_star[j * Nx + i]);
        ru_new[j * Nx + i] = 0.5 * ru[j * Nx + i] + 0.5 * (ru_star[j * Nx + i] + dt * rhs_ru_star[j * Nx + i]);
        rv_new[j * Nx + i] = 0.5 * rv[j * Nx + i] + 0.5 * (rv_star[j * Nx + i] + dt * rhs_rv_star[j * Nx + i]);
        e_new[j * Nx + i] = 0.5 * e[j * Nx + i] + 0.5 * (e_star[j * Nx + i] + dt * rhs_e_star[j * Nx + i]);
    }
}

void rk2_step(double *r, double *ru, double *rv, double *e,
             double *p, double dt,
             double *r_new, double *ru_new, double *rv_new, double *e_new, double *p_new) {
    double k1, k2, k3;
    double *rhs_r, *rhs_ru, *rhs_rv, *rhs_e;
    double *r_star, *ru_star, *rv_star, *e_star, *p_star;
    double *rhs_r_star, *rhs_ru_star, *rhs_rv_star, *rhs_e_star;

    size_t array_size = Nx * Ny * sizeof(double);

    cudaMalloc(&rhs_r, array_size);
    cudaMalloc(&rhs_ru, array_size);
    cudaMalloc(&rhs_rv, array_size);
    cudaMalloc(&rhs_e, array_size);
    cudaMalloc(&r_star, array_size);
    cudaMalloc(&ru_star, array_size);
    cudaMalloc(&rv_star, array_size);
    cudaMalloc(&e_star, array_size);
    cudaMalloc(&p_star, array_size);
    cudaMalloc(&rhs_r_star, array_size);
    cudaMalloc(&rhs_ru_star, array_size);
    cudaMalloc(&rhs_rv_star, array_size);
    cudaMalloc(&rhs_e_star, array_size);
    cudaCheckError();

    dim3 blockDim(BLOCK_SIZE_X, BLOCK_SIZE_Y);
    dim3 gridDim((Nx + blockDim.x - 1) / blockDim.x, (Ny + blockDim.y - 1) / blockDim.y);

    update_diffusion_coeffs(dt, &k1, &k2, &k3);
    compute_rhs(r, ru, rv, e, p, k1, k2, k3, rhs_r, rhs_ru, rhs_rv, rhs_e);

    rk2_step_kernel1<<<gridDim, blockDim>>>(r, ru, rv, e, rhs_r, rhs_ru, rhs_rv, rhs_e,
                                          r_star, ru_star, rv_star, e_star, dt);
    cudaDeviceSynchronize();
    cudaCheckError();

    update_pressure(r_star, ru_star, rv_star, e_star, p_star);
    apply_boundary_conditions(r_star, ru_star, rv_star, e_star, p_star);

    compute_rhs(r_star, ru_star, rv_star, e_star, p_star, k1, k2, k3,
               rhs_r_star, rhs_ru_star, rhs_rv_star, rhs_e_star);

    rk2_step_kernel2<<<gridDim, blockDim>>>(r, ru, rv, e, r_star, ru_star, rv_star, e_star,
                                          rhs_r_star, rhs_ru_star, rhs_rv_star, rhs_e_star,
                                          r_new, ru_new, rv_new, e_new, dt);
    cudaDeviceSynchronize();
    cudaCheckError();

    update_pressure(r_new, ru_new, rv_new, e_new, p_new);
    apply_boundary_conditions(r_new, ru_new, rv_new, e_new, p_new);

    update_state_kernel<<<gridDim, blockDim>>>(r, ru, rv, e, p, r_new, ru_new, rv_new, e_new, p_new);
    cudaDeviceSynchronize();
    cudaCheckError();

    cudaFree(rhs_r);
    cudaFree(rhs_ru);
    cudaFree(rhs_rv);
    cudaFree(rhs_e);
    cudaFree(r_star);
    cudaFree(ru_star);
    cudaFree(rv_star);
    cudaFree(e_star);
    cudaFree(p_star);
    cudaFree(rhs_r_star);
    cudaFree(rhs_ru_star);
    cudaFree(rhs_rv_star);
    cudaFree(rhs_e_star);
}

void euler_step(double *r, double *ru, double *rv, double *e,
              double *p, double dt,
              double *r_new, double *ru_new, double *rv_new, double *e_new, double *p_new) {
    double k1, k2, k3;
    double *rhs_r, *rhs_ru, *rhs_rv, *rhs_e;

    cudaMalloc(&rhs_r, Nx * Ny * sizeof(double));
    cudaMalloc(&rhs_ru, Nx * Ny * sizeof(double));
    cudaMalloc(&rhs_rv, Nx * Ny * sizeof(double));
    cudaMalloc(&rhs_e, Nx * Ny * sizeof(double));
    cudaCheckError();

    dim3 blockDim(BLOCK_SIZE_X, BLOCK_SIZE_Y);
    dim3 gridDim((Nx + blockDim.x - 1) / blockDim.x, (Ny + blockDim.y - 1) / blockDim.y);

    update_diffusion_coeffs(dt, &k1, &k2, &k3);
    compute_rhs(r, ru, rv, e, p, k1, k2, k3, rhs_r, rhs_ru, rhs_rv, rhs_e);

    euler_step_kernel<<<gridDim, blockDim>>>(r, ru, rv, e, rhs_r, rhs_ru, rhs_rv, rhs_e,
                                           r_new, ru_new, rv_new, e_new, dt);
    cudaDeviceSynchronize();
    cudaCheckError();

    update_pressure(r_new, ru_new, rv_new, e_new, p_new);
    apply_boundary_conditions(r_new, ru_new, rv_new, e_new, p_new);

    update_state_kernel<<<gridDim, blockDim>>>(r, ru, rv, e, p, r_new, ru_new, rv_new, e_new, p_new);
    cudaDeviceSynchronize();
    cudaCheckError();

    cudaFree(rhs_r);
    cudaFree(rhs_ru);
    cudaFree(rhs_rv);
    cudaFree(rhs_e);
}

// Function to save density to file
void save_density(double *d_r, int step) {
    double *h_r = (double*)malloc(Nx * Ny * sizeof(double));
    char filename[256];
    snprintf(filename, sizeof(filename), "density/density_%06d.dat", step);

    cudaMemcpy(h_r, d_r, Nx * Ny * sizeof(double), cudaMemcpyDeviceToHost);
    cudaCheckError();

    FILE *fp = fopen(filename, "w");
    if (fp == NULL) {
        printf("Error opening file %s\n", filename);
        exit(EXIT_FAILURE);
    }

    for (int j = 0; j < Ny; j++) {
        for (int i = 0; i < Nx; i++) {
            fprintf(fp, "%e ", h_r[j * Nx + i]);
        }
        fprintf(fp, "\n");
    }

    fclose(fp);
    free(h_r);
}

int main() {
    // Device pointers
    double *d_r, *d_ru, *d_rv, *d_e, *d_p;
    double *d_r_new, *d_ru_new, *d_rv_new, *d_e_new, *d_p_new;
    size_t array_size = Nx * Ny * sizeof(double);

    // Allocate device memory
    cudaMalloc(&d_r, array_size);
    cudaMalloc(&d_ru, array_size);
    cudaMalloc(&d_rv, array_size);
    cudaMalloc(&d_e, array_size);
    cudaMalloc(&d_p, array_size);
    cudaMalloc(&d_r_new, array_size);
    cudaMalloc(&d_ru_new, array_size);
    cudaMalloc(&d_rv_new, array_size);
    cudaMalloc(&d_e_new, array_size);
    cudaMalloc(&d_p_new, array_size);
    cudaCheckError();

    // Simulation parameters
    double t = 0.0;
    double max_time = 7.0;
    double dt;
    int step = 0;
    int use_rk2 = 1; // Use RK2 method

    // Create output directory
    system("rm -rf density");
    mkdir("density", 0755);

    // Initialize grid
    initialize_grid(d_r, d_ru, d_rv, d_e, d_p);
    save_density(d_r, 0);

    // Time-stepping loop
    while (t < max_time) {
        dt = compute_dt(d_r, d_ru, d_rv, d_p);
        if (t + dt > max_time) {
            dt = max_time - t;
        }

        if (use_rk2) {
            rk2_step(d_r, d_ru, d_rv, d_e, d_p, dt, d_r_new, d_ru_new, d_rv_new, d_e_new, d_p_new);
        } else {
            euler_step(d_r, d_ru, d_rv, d_e, d_p, dt, d_r_new, d_ru_new, d_rv_new, d_e_new, d_p_new);
        }

        t += dt;
        step++;

        if (step % save_interval == 0) {
            printf("Step %d: t = %.4f, dt = %.4f\n", step, t, dt);
            save_density(d_r, step);
        }
    }

    // Save final state
    save_density(d_r, step);

    // Free device memory
    cudaFree(d_r);
    cudaFree(d_ru);
    cudaFree(d_rv);
    cudaFree(d_e);
    cudaFree(d_p);
    cudaFree(d_r_new);
    cudaFree(d_ru_new);
    cudaFree(d_rv_new);
    cudaFree(d_e_new);
    cudaFree(d_p_new);

    return 0;
}


Writing rt_simulation.cu


In [3]:
!nvcc rt_simulation.cu -o rt_simulation


In [4]:
!./rt_simulation


Step 1000: t = 0.0467, dt = 0.0000
Step 2000: t = 0.0934, dt = 0.0000
Step 3000: t = 0.1402, dt = 0.0000
Step 4000: t = 0.1869, dt = 0.0000
Step 5000: t = 0.2336, dt = 0.0000
Step 6000: t = 0.2804, dt = 0.0000
Step 7000: t = 0.3271, dt = 0.0000
Step 8000: t = 0.3739, dt = 0.0000
Step 9000: t = 0.4206, dt = 0.0000
Step 10000: t = 0.4674, dt = 0.0000
Step 11000: t = 0.5141, dt = 0.0000
Step 12000: t = 0.5609, dt = 0.0000
Step 13000: t = 0.6076, dt = 0.0000
Step 14000: t = 0.6544, dt = 0.0000
Step 15000: t = 0.7012, dt = 0.0000
Step 16000: t = 0.7479, dt = 0.0000
Step 17000: t = 0.7947, dt = 0.0000
Step 18000: t = 0.8415, dt = 0.0000
Step 19000: t = 0.8883, dt = 0.0000
Step 20000: t = 0.9351, dt = 0.0000
Step 21000: t = 0.9818, dt = 0.0000
Step 22000: t = 1.0286, dt = 0.0000
Step 23000: t = 1.0754, dt = 0.0000
Step 24000: t = 1.1222, dt = 0.0000
Step 25000: t = 1.1690, dt = 0.0000
Step 26000: t = 1.2158, dt = 0.0000
Step 27000: t = 1.2626, dt = 0.0000
Step 28000: t = 1.3093, dt = 0.0000
S

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

# Parameters
Nx = 512
Ny = 1024
Lx = 1.0
Ly = 2.0
max_time = 5.0          # Match CUDA's max_time
save_interval = 1000    # Match CUDA's save_interva

density_dir = "density"

density_files = [f for f in os.listdir(density_dir) if f.startswith("density_") and f.endswith(".dat")]
steps = [int(f.split("_")[1].split(".")[0]) for f in density_files]
files_and_steps = sorted(zip(density_files, steps), key=lambda x: x[1])

density_files = [f for f, _ in files_and_steps]
steps = [s for _, s in files_and_steps]

if not density_files:
    raise FileNotFoundError(f"No density files found in {density_dir}")

fig, ax = plt.subplots(figsize=(4, 8))  
x = np.linspace(0, Lx, Nx)
y = np.linspace(0, Ly, Ny)
X, Y = np.meshgrid(x, y)

initial_data = np.loadtxt(os.path.join(density_dir, density_files[0]))  
im = ax.pcolormesh(X, Y, initial_data, cmap='viridis', shading='nearest', vmin=1.0, vmax=2.0)
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_aspect('equal')
ax.set_title(f"Step {steps[0]}, t ≈ 0.000")
fig.colorbar(im, ax=ax, label='Density')

def update(frame):
    filepath = os.path.join(density_dir, density_files[frame])
    density = np.loadtxt(filepath)  # Shape (Ny, Nx) = (128, 64)
    im.set_array(density.ravel())  # Flatten to 1D array for pcolormesh
    time = steps[frame] * max_time / max(steps) if steps else 0.0
    ax.set_title(f"Step {steps[frame]}, t ≈ {time:.3f}")
    return [im]

ani = FuncAnimation(fig, update, frames=len(density_files), interval=100, blit=True)

ani.save('density_animation.mp4', writer='ffmpeg', fps=10)

plt.close(fig)